# A toy example based on the codelab here: 
#### https://codelabs.developers.google.com/vertex-pipelines-intro

In [1]:
USER_FLAG = "--user"

In [ ]:
!pip3 install {USER_FLAG} google-cloud-aiplatform --upgrade
!pip3 install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [2]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.7.0
google_cloud_pipeline_components version: 0.1.4


### KFP SDK v2 documentation:
#### https://www.kubeflow.org/docs/components/pipelines/sdk/v2/
### KFP Github:
#### https://github.com/kubeflow/pipelines
### GCPC documentation:
#### https://google-cloud-pipeline-components.readthedocs.io/
### GCPC Github:
#### https://github.com/kubeflow/pipelines/tree/master/components/google-cloud

In [3]:
import os
PROJECT_ID = "kubeflow-on-gcp-123"

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  kubeflow-on-gcp-123


In [4]:
BUCKET_NAME="gs://aiplatform-custom"

### Import a number of packages with comments describing each

In [5]:
from typing import NamedTuple

import kfp
from kfp import dsl # contains the domain-specific language (DSL) that you can use to define and interact with pipelines and components
from kfp.v2 import compiler # includes classes and methods for compiling pipeline Python DSL into a workflow JSON spec
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, ClassificationMetrics, Metrics, component) #import a number of features from the v2 DSL
from kfp.v2.google.client import AIPlatformClient # client used to interface with the Vertex AI APIs - to be deprecated

from google.cloud import aiplatform # Vertex AI SDK - new interface to interface with APIS programmatically
from google_cloud_pipeline_components import aiplatform as gcc_aip # pre-built components for Vertex AI

In [6]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://aiplatform-custom/pipeline_root/'

#### A Python function based component
##### @component is a decorator used to compile the function when the pipeline runs - used anytime a custom component is defined
##### base_image parameter specifies the container image this component will use - in this case, an image from public DockerHub for Python 3.9; default is Python 3.7
##### output_component_file parameter is optional and specifies the yaml file to write the compiled component to

In [7]:
@component(base_image="python:3.9", output_component_file="first-component.yaml")
def product_name(text: str) -> str: # all pipeline parameters must be annotated with data types
    return text

In [ ]:
# product_name_component = kfp.components.load_component_from_file('./first-component.yaml')

##### packages_to_install paramater tells the component any external library dependencies for this container

In [8]:
@component(packages_to_install=["emoji"])
def emoji(
    text: str,
) -> NamedTuple(
    "Outputs",
    [
        ("emoji_text", str),
        ("emoji", str),
    ],
):
    import emoji

    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', use_aliases=True)
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    return (emoji_text, emoji_str)

In [9]:
@component
def build_sentence(
    product: str, #output from previous task
    emoji: str, #output from previous task
    emojitext: str #output from previous task
) -> str:
    print("We completed the pipeline, hooray!")
    end_str = product + " is "
    if len(emoji) > 0:
        end_str += emoji
    else:
        end_str += emojitext
    return(end_str)

#### Define a pipeline function
##### @dsl.pipeline decorator with arguments

In [10]:
@dsl.pipeline(
    name="emoji-pipeline",
    description="An intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)

# Change the `text` and `emoji_str` parameters to update the pipeline output
def intro_pipeline(text: str = "Vertex Model Monitoring", emoji_str: str = "sparkles"):
    product_task = product_name(text) # .set_caching_options(False)
    emoji_task = emoji(emoji_str)
    consumer_task = build_sentence(
        product_task.output,
        emoji_task.outputs["emoji"],
        emoji_task.outputs["emoji_text"],
    )

#### This is the pipeline BUILD step
##### can also be compiled with dsl-compile-v2 --py <py_file> --output <output_json>, this would require an actual .py application

In [11]:
compiler.Compiler().compile(
    pipeline_func=intro_pipeline, package_path="emoji_pipeline_spec.json"
)

#### This is the pipeline DEPLOY step
##### can also be done with the Vertex AI SDK as seen in the commented cells below
##### in KFP v1 the CLI can 'upload' pipeline specs and 'submit run' to create a run, not yet available with v2
##### https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline#creating_a_pipeline_run

In [15]:
api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
)

response = api_client.create_run_from_job_spec(
    job_spec_path="emoji_pipeline_spec.json",
    enable_caching=True) # True by default
    # 'KFP caching' checks to see whether or not an execution exists in ML Metadata with the interface of each pipeline step
    # if there is a matching execution the outputs of that execution are used and the step is skipped

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:175: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


In [ ]:
#from google.cloud import aiplatform

#pipeline_job = aiplatform.PipelineJob(
#   display_name='emoji-pipeline',
#   template_path='emoji_pipeline_spec.json',
#   pipeline_root=PIPELINE_ROOT,
#   enable_caching=True
#)

In [ ]:
#pipeline_job.run(sync=False)

In [ ]:
#aiplatform.PipelineJob.list()

In [ ]:
#pipeline_job.delete()